In [ ]:
def generate_sh(study_name):
    path = f"../fill-gaps-configs/{study_name}-fill-gaps.sh"
    script = f"""#!/bin/sh
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem-per-cpu=4G
#SBATCH --time=300:0:0
#SBATCH --output={study_name}.out
#SBATCH --error={study_name}.err

qptuna-build --config {study_name}-fill-gaps.json --best-model-outpath ../best-models/best-{study_name}.pkl
"""
    
    with open(path, 'w') as f:
        f.write(script)


In [ ]:
def build_model(study_name):
    file = f"../optuna-storage/optuna_storage_{study_name}.sqlite"
    storage = f"sqlite:///{os.path.abspath(file)}"
    
    try:
        loaded_study = optuna.load_study(
            study_name=study_name, 
            storage=storage
        )
    except Exception as e:
        print(f"Could not open study {study_name} from storage {storage}: {e}")
        return collections.defaultdict(lambda: np.nan)  # All scores NaN.

    buildconfig = buildconfig_best(loaded_study)
    
    # Change training set path.
    buildconfig.data.training_dataset_file = "".join(["../", buildconfig.data.training_dataset_file])
    
    # Save build config.
    with open(f"../fill-gaps-configs/{study_name}-fill-gaps.json", 'w') as f:
        json.dump(json.loads(buildconfig.json()), f, indent=4)
    
    # Save shell script for SLURM submission.
    generate_sh(study_name)


In [ ]:
build_model("MMP_data_Solubility_set3_xgboost")
build_model("MMP_data_logD_set3_xgboost")
build_model("MMP_data_logD_set3_RF")

build_model("MMP_data_logD_set1_xgboost")
build_model("MMP_data_Clearance_set4_SVR")
build_model("MMP_data_Clearance_set3_xgboost")
build_model("MMP_data_Clearance_set1_xgboost")
build_model("MMP_data_Solubility_set1_xgboost")
build_model("MMP_data_Clearance_set3_RF")
build_model("MMP_data_Clearance_set1_RF")
build_model("MMP_data_Solubility_set3_RF")
build_model("MMP_data_Solubility_set1_RF")
build_model("MMP_data_logD_set1_RF")
build_model("MMP_data_logD_set4_SVR")
build_model("MMP_data_Solubility_set4_SVR")

build_model("MMP_downsampled-10-percent_logD_set3_SVR")
build_model("MMP_downsampled-10-percent_Solubility_set3_SVR")
build_model("MMP_downsampled-10-percent_logD_set1_SVR")
build_model("MMP_downsampled-10-percent_Solubility_set1_SVR")